In [3]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [4]:
train_df = pd.read_csv('input/train.csv')
# train_df = pd.read_csv('input/train_min.csv')
test_df = pd.read_csv('input/test.csv')

X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [9]:
n_splits = 5
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [10]:
# cv: 0.89951
params = {
         'num_leaves': 8,
         'min_data_in_leaf': 42,
         'objective': 'binary',
         'max_depth': 16,
         'learning_rate': 0.0123,
         'boosting': 'gbdt',
         'bagging_freq': 5,
         'bagging_fraction': 0.8,
         'feature_fraction': 0.8201,
         'bagging_seed': 11,
         'reg_alpha': 1.728910519108444,
         'reg_lambda': 4.9847051755586085,
         'random_state': 42,
         'metric': 'auc',
         'verbosity': -1,
         'subsample': 0.81,
         'min_gain_to_split': 0.01077313523861969,
         'min_child_weight': 19.428902804238373,
         'num_threads': 4 
        }

'''
# cv: 0.90775 (full)
# lb: 0.891 with fold_n=2 => overfit?
params = {
         'objective': 'binary',
         'max_depth': 127,
         'metric': 'auc',
         'boosting':'gbdt',
         }
'''

# 1st round looks good: cv 0.900043
# lb: 0.899 with n_splits=10
# try larger max_bin?
params = {
         'num_leaves': 8,
         'min_data_in_leaf': 1000,
         'objective': 'binary',
         'max_depth': -1,
         'learning_rate': 0.0123,
         'boosting': 'gbdt',
         'bagging_freq': 5,
         'bagging_fraction': 0.8,
         'feature_fraction': 0.8201,
         'bagging_seed': 11,
         'reg_alpha': 1.728910519108444,
         'reg_lambda': 4.9847051755586085,
         'random_state': 42,
         'metric': 'auc',
         'verbosity': -1,
         'subsample': 0.81,
         'min_gain_to_split': 0.01077313523861969,
         'min_child_weight': 19.428902804238373,
         'num_threads': 4,
         'max_bin': 511,
        }


In [11]:
prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    model = lgb.train(params,train_data,num_boost_round=200000,
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 200)
    
    prediction += model.predict(X_test, num_iteration=model.best_iteration)/n_splits

Fold 0 started at Fri Mar  1 23:58:01 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.828386	valid_1's auc: 0.814217
[600]	training's auc: 0.863604	valid_1's auc: 0.845652
[900]	training's auc: 0.881215	valid_1's auc: 0.860598
[1200]	training's auc: 0.892594	valid_1's auc: 0.869657
[1500]	training's auc: 0.90054	valid_1's auc: 0.875986
[1800]	training's auc: 0.906721	valid_1's auc: 0.880307
[2100]	training's auc: 0.911733	valid_1's auc: 0.883937
[2400]	training's auc: 0.915626	valid_1's auc: 0.886663
[2700]	training's auc: 0.918834	valid_1's auc: 0.888649
[3000]	training's auc: 0.921659	valid_1's auc: 0.890267
[3300]	training's auc: 0.924027	valid_1's auc: 0.891784
[3600]	training's auc: 0.926181	valid_1's auc: 0.893028
[3900]	training's auc: 0.928002	valid_1's auc: 0.894059
[4200]	training's auc: 0.92959	valid_1's auc: 0.894852
[4500]	training's auc: 0.931071	valid_1's auc: 0.895553
[4800]	training's auc: 0.932433	valid_1's auc: 0.896153
[51

[8700]	training's auc: 0.946437	valid_1's auc: 0.895342
Early stopping, best iteration is:
[8578]	training's auc: 0.94603	valid_1's auc: 0.895379


In [12]:
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = prediction
sub.to_csv("submission.csv", index=False)